# Template Migration to ORM
This notebook provides the steps to migrate a database from the prior `problem_instances` and `trials` tables to the new SQLAlchemy ORM based tables. 

In [ ]:
import datetime

from benchmarklib import BenchmarkDatabase, _BenchmarkDatabase
from benchmarklib import BaseProblem, BaseTrial
from benchmarklib.core.types import _ProblemInstance, _BaseTrial


Change rbf references in the following two cells to the problem type you are migrating to the ORM.

In [ ]:
from rbf import RandomBooleanFunction, RandomBooleanFunctionTrial, _RandomBooleanFunction, _RandomBooleanFunctionTrial
old_db = _BenchmarkDatabase("rbf.db", _RandomBooleanFunction, _RandomBooleanFunctionTrial)
new_db = BenchmarkDatabase("rbf_new.db", RandomBooleanFunction, RandomBooleanFunctionTrial)

In [ ]:
# specify the construction of the new object types from the old ones in this cell

def upgrade_problem(old_problem: _ProblemInstance) -> BaseProblem:
    """
    return a new problem instance constructed from the old problem instance
    """
    return RandomBooleanFunction(
        num_vars=old_problem.num_vars,
        complexity=old_problem.complexity,
        statement=old_problem.statement,
    )

def upgrade_trial(old_trial: _BaseTrial, new_problem: BaseProblem) -> BaseTrial:
    """
    return a new trial instance constructed from the old trial instance and its corresponding new problem instance
    """
    # convert to new dedicated is_failed attribute to keep the meaning of counts consistent
    is_failed = old_trial.counts is not None and "-1" in old_trial.counts
    counts = None if is_failed else old_trial.counts

    return RandomBooleanFunctionTrial(
        problem=new_problem,
        compiler_name=old_trial.compiler_name,
        job_id=old_trial.job_id,
        job_pub_idx=old_trial.job_pub_idx,
        counts=counts,
        simulation_counts=old_trial.simulation_counts,
        is_failed=is_failed,
        input_state=old_trial.trial_params.get("input_state"),
        created_at=datetime.datetime.fromisoformat(old_trial.created_at)
    )

In [ ]:
# keep track of completions and errors here, in case the following cell needs to be run multiple times
completed = set()
errors = set()

In [ ]:
# run this cell to perform the migration

with old_db._connect() as conn:
    last_instance_id = conn.execute("SELECT MAX(instance_id) FROM problem_instances").fetchone()[0]

for instance_id in range(1, last_instance_id + 1):
    if instance_id in completed:
        continue

    try:
        old_problem = old_db.get_problem_instance(instance_id)
    except ValueError:
        completed.add(instance_id)
        continue

    old_trials = old_db.find_trials(instance_id=instance_id)
    new_problem = upgrade_problem(old_problem)

    with new_db.session() as session:
        try:
            for old_trial in old_trials:
                session.add(new_problem)
                session.flush()  # to get new_problem.instance_id

                new_trial = upgrade_trial(old_trial, new_problem)
                session.add(new_trial)

            session.commit()
            completed.add(instance_id)
            
        except Exception as e:
            session.rollback()
            errors.add(instance_id)
            print(f"Error migrating instance_id {instance_id}: {e}")


In [ ]:
# sanity check that we have the same number of problems and trials

from sqlalchemy import select, func

with old_db._connect() as conn:
    total_problems = conn.execute("SELECT COUNT(*) FROM problem_instances").fetchone()[0]
    total_trials = conn.execute("SELECT COUNT(*) FROM trials").fetchone()[0]

print(f"Total problems in old DB: {total_problems}")
print(f"Total trials in old DB: {total_trials}")

with new_db.session() as session:
    query = select(func.count(RandomBooleanFunction.id))
    total_problems = session.execute(query).scalar_one()

    query = select(func.count(RandomBooleanFunctionTrial.id))
    total_trials = session.execute(query).scalar_one()

print(f"Total problems in new DB: {total_problems}")
print(f"Total trials in new DB: {total_trials}")

Total problems in new DB: 6993
Total trials in new DB: 725886


rename the databases if everything looks good
```bash
mv rbf.db _rbf.db
mv rbf_new.db rbf.db
```
And you can delete `_rbf.db` after verifying it contains all the necessary data.